# Import Libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import os, sys
from importlib import reload
from config import data_config
from utils.args_parser import add_data_args

# Reload arg_parser because Jupyter notebook does not reload it automatically
reload(data_config)

# Add arguments to parser
parser = ArgumentParser()
parser = add_data_args(parser)

# Print default dataset parameters
args = parser.parse_args([])
print("\n=====================")
print("Default dataset parameters:")
for arg in vars(args):
    print(arg, '=', getattr(args, arg))
print("\n=====================")




Default dataset parameters:
ode_system = pendulum
t_max = 20
step_size = 0.01
n_sample = 10
x_init_pts = [[-3.14159265  3.14159265]
 [-8.          8.        ]]
ctr_func = gaussian
datafile_path = data/pendulum_ctr_gaussian_N_10_T20.npy



In [4]:
# Run database_generator.py with arguments
print("\n=====================")
print("Running database_generator.py with arguments:")
%run database_generator.py --n_sample 100 --ctr_func "gaussian" --datafile_path "data/pendulum_ctr_grf_N_100_T20.npy"



Running database_generator.py with arguments:


Generating 100 data ...:  19%|█▉        | 19/100 [00:04<00:18,  4.45it/s]/PENDULUM/src/config/data_config.py:35: RuntimeWarning: invalid value encountered in subtract
  u - b * omega - 0.5 * m * l * g * np.sin(theta)
/PENDULUM/src/config/data_config.py:35: RuntimeWarning: invalid value encountered in sin
  u - b * omega - 0.5 * m * l * g * np.sin(theta)
Generating 100 data ...: 100%|██████████| 100/100 [00:24<00:00,  4.02it/s]

Data saved in data/pendulum_ctr_grf_N_100_T20.npy .


In [ ]:
%run train.py --datafile_path "data/pendulum_ctr_grf_N_5000_T20.npy" --search_num 2 --epochs 100